In [1]:
import sys

In [2]:
print(sys.path)

['/Users/balarusum/Desktop/Bio metrics', '/Users/balarusum/opt/anaconda3/lib/python39.zip', '/Users/balarusum/opt/anaconda3/lib/python3.9', '/Users/balarusum/opt/anaconda3/lib/python3.9/lib-dynload', '', '/Users/balarusum/opt/anaconda3/lib/python3.9/site-packages', '/Users/balarusum/opt/anaconda3/lib/python3.9/site-packages/aeosa']


In [3]:
sys.path.append('/Users/balarusum/Library/Python/3.9/lib/python/site-packages')

In [4]:
import cv2
import mediapipe as mp


In [5]:
# Load Mediapipe solutions
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face_detection = mp.solutions.face_detection

In [6]:
# Initialize Mediapipe solutions
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5)
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
video = cv2.VideoCapture('Video.mp4')
# Get the video height and width
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))

[ WARN:0@3.255] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@3.255] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@3.255] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@3.255] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_16645483

In [ ]:
SHRUG_THRESHOLD = 0.6
# Loop over the frames in the video
while True:
    # Read the next frame
    ret, frame = video.read()
    if not ret:
        break

    # Convert the frame to RGB format
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe solutions
    results_hands = hands.process(frame)
    results_pose = pose.process(frame)
    results_face_detection = face_detection.process(frame)

    # Draw bounding boxes on hands
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            x_min, y_min, x_max, y_max = w, h, 0, 0
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * w), int(landmark.y * h)
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

    # Draw bounding boxes on faces
    if results_face_detection.detections:
        for detection in results_face_detection.detections:
            x_min, y_min, x_max, y_max = w, h, 0, 0
            bbox = detection.location_data.relative_bounding_box
            x_min = int(bbox.xmin * w)
            y_min = int(bbox.ymin * h)
            x_max = int((bbox.xmin + bbox.width) * w)
            y_max = int((bbox.ymin + bbox.height) * h)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
     # Draw straight lines on shoulders
    if results_pose.pose_landmarks:
        left_shoulder_x = int(results_pose.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w)
        left_shoulder_y = int(results_pose.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h)
        right_shoulder_x = int(results_pose.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w)
        right_shoulder_y = int(results_pose.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h)
        #cv2.line(frame, (0, left_shoulder_y), (left_shoulder_x - 10, left_shoulder_y), (0, 255, 0), 2)
        #cv2.line(frame, (right_shoulder_x + 10, right_shoulder_y), (w, right_shoulder_y), (0, 255, 0), 2)
        
        # Calculate the distance between the shoulders
        shoulder_distance = right_shoulder_x - left_shoulder_x
        
        # Calculate the midpoint between the left and right shoulders
        midpoint_x = (left_shoulder_x + right_shoulder_x) // 2
        midpoint_y = (left_shoulder_y + right_shoulder_y) // 2
        
        

        line_y = int(midpoint_y + 0.2 * shoulder_distance)
        cv2.line(frame, (left_shoulder_x, line_y), (right_shoulder_x, line_y), (0, 255, 0), 2)

        
        # Check for shrug
        if left_shoulder_y < h * SHRUG_THRESHOLD and right_shoulder_y < h * SHRUG_THRESHOLD:
            cv2.putText(frame, "Shrug detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow("Video", frame)

    # Exit if the user presses 'q'
    if cv2.waitKey(1) == ord("q"):
        break

# Clean up
video.release()

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)